In [ ]:
%%writefile data/p3d.py
import math
import numpy as np

from sailfish.subdomain import Subdomain3D
from sailfish.node_type import NTFullBBWall, NTEquilibriumDensity
from sailfish.controller import LBSimulationController
from sailfish.lb_single import LBFluidSim


class PoiseuilleSubdomain(Subdomain3D):
    """simple 3D Poiseuille geometry."""

    wall_bc = NTFullBBWall

    def _set_pressure_bc(self, hx, hy, hz, wall_map):
        pressure_bc = NTEquilibriumDensity
        not_wall = np.logical_not(wall_map)
    
        inlet_map = (hx == 0) & not_wall
        outlet_map = (hx == self.gx - 1) & not_wall

        self.set_node(inlet_map, pressure_bc(1.0 + self.config.delta_rho / 2.0))
        self.set_node(outlet_map, pressure_bc(1.0 - self.config.delta_rho / 2.0))

    def boundary_conditions(self, hx, hy, hz):
        
        radiussq = (self.channel_width(self.config) / 2.0)**2

        wall_map = (hy - (self.gy / 2 - 0.5))**2 + (hz - (self.gz / 2 - 0.5))**2 >= radiussq
        np.savez("tube",wall_map=wall_map)
        self.set_node(wall_map, self.wall_bc)
        self._set_pressure_bc(hx, hy, hz, wall_map)

    def initial_conditions(self, sim, hx, hy, hz):
        sim.rho[:] = 1.0
        self.config.logger.info('MK Starting, l0=%f'%self.channel_width(self.config))

    @classmethod
    def channel_width(cls, config):
        return cls.width(config) - 1 - 2 * cls.wall_bc.location

    @classmethod
    def width(cls, config):
        return min(config.lat_ny, config.lat_nz)
        

class PoiseuilleSim(LBFluidSim):
    subdomain = PoiseuilleSubdomain

    @classmethod
    def update_defaults(cls, defaults):
        defaults.update({
            'grid': 'D3Q19',
            'lat_nx': 64,
            'lat_ny': 64,
            'lat_nz': 64,
            'visc': 0.1,
            'delta_rho': 0.01234,
            'debug_single_process':True,
            'max_iters':1000,
             'verbose':0
            })

    @classmethod
    def add_options(cls, group, dim):
        group.add_argument('--delta_rho', type=float, default=0.001,help='Difference in density across the system in LB units.')

    def __init__(self, config):
        super(PoiseuilleSim, self).__init__(config)

        
if __name__ == '__main__':
    ctrl = LBSimulationController(PoiseuilleSim)
    ctrl.run()

In [ ]:
%%sh
rm data/p3d*npz
export PYTHONPATH=$PWD/sailfish/ 
python data/p3d.py --precision=double --max_iters=30000 --every=10000 --from=0 --output=data/p3d --output_format=npy\
 --lat_nx=64 --lat_ny=64 --lat_nz=64 --visc=0.055625 --delta_rho=0.000434090322581

In [1]:
data = np.load('data/p3d.0.30000.npz')
vx,vy,vz  = data['v']

NameError: name 'np' is not defined

In [ ]:
import glob
from IPython.display import display
casename="p3d"
casefiles =sorted( glob.glob("data/"+casename+"*.*[0-9].npz") )

for fn in casefiles:
    data = np.load(fn)
    vx,vy,vz  = data['v']
    print fn,"--> Re from LBM:",np.nanmax(vx[:,:,:])*ny/visc,"u_max=",np.nanmax(vx[:,:,:])
    


In [ ]:
from k3d import K3D
import numpy as np 
color_map = (0x0000ff, 0xff0000,0x00ff00)
color_map2 = (0x996600, 0xffff00,0x0f10ff)
plot = K3D()
gx,gy,gz= tube.shape
s = 10.

M = np.array([[  0.   ,   0. ,  -gx/s, gx/(s*2)],
              [  0.   ,  -gy/s,   0. ,  gy/(s*2)],
              [  -gz/s ,   0. ,   0. ,  gz/(s*2)],
              [  0.   ,   0. ,   0. ,  1.] ], dtype=np.float32)


tube = np.load('tube.npz')['wall_map']

walls  = tube
geom_walls = K3D.voxels( walls, color_map2,view_matrix=M)
vec = K3D.vectors(
   ( 0,0,0,0, 0,0,0,0,0), 
   (30,0,0,0,20,0,0,0,10), 
   colors=(0xff0000, 0xff0000, 0x0000ff, 0x0000ff, 0x00ff00, 0x00ff00), 
   labels=('Axis x', 'Axis y', 'Axis z',"K")
)
plot += vec
plot.display()
plot.camera_auto_fit = False
plot += geom_walls

In [2]:
print "--lat_nx="+str(nx)+" --lat_ny="+str(ny)+" --lat_nz="+str(nz)+" --visc="+str(visc)+\
 " --delta_rho="+str(delta_rho_lu)

NameError: name 'nx' is not defined